In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
# Load the data
df = pd.read_csv('datasets/dataset_L.csv')

# Separate features and labels
X = df.iloc[:, :-7].values  # First 170 columns as features
y = df.iloc[:, -7:].values  # Last 7 columns as labels

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)


In [4]:
import torch.nn.functional as F

class NetDropout(nn.Module):
    def __init__(self, n=32):
        super(NetDropout, self).__init__()
        self.n = n
        
        self.conv1 = nn.Conv1d(1, self.n, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout1d(p=0.3)
        self.conv2 = nn.Conv1d(self.n, self.n // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout1d(p=0.3)
        self.conv3 = nn.Conv1d(self.n // 2, self.n // 2, kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout1d(p=0.3)

        # Calculate the correct input size for the fully connected layer
        self.fc1 = nn.Linear((self.n // 2) * (170 // 8), 32)  # 170 // 8 due to three max_pool1d with kernel_size=2
        self.fc2 = nn.Linear(32, 7)
        
        self.lsftmx = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = F.max_pool1d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)

        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = torch.tanh(self.fc1(out))
        out = self.lsftmx(self.fc2(out))
        return out


In [3]:
import torch.nn.functional as F

class NetDropoutBig(nn.Module):
    def __init__(self, n=32):
        super(NetDropoutBig, self).__init__()
        self.n = n
        
        self.conv1 = nn.Conv1d(1, self.n, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout1d(p=0.3)
        self.conv2 = nn.Conv1d(self.n, self.n // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout1d(p=0.3)
        self.conv3 = nn.Conv1d(self.n // 2, self.n // 2, kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout1d(p=0.3)
        self.conv4 = nn.Conv1d(self.n // 2, self.n // 4, kernel_size=3, padding=1)
        self.conv4_dropout = nn.Dropout1d(p=0.3)

        # Calculate the correct input size for the fully connected layer
        self.fc1 = nn.Linear((self.n // 4) * (170 // 16), 32)  # 170 // 16 due to four max_pool1d with kernel_size=2
        self.fc2 = nn.Linear(32, 7)
        
        self.lsftmx = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = F.max_pool1d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv4(out)), 2)
        out = self.conv4_dropout(out)

        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = torch.tanh(self.fc1(out))
        out = self.lsftmx(self.fc2(out))
        return out


In [4]:
# 80% training and 20% testing split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Initialize the model, loss function, and optimizer
model = NetDropout().to(device)
# model = NetDropoutBig().to(device)
criterion = nn.CrossEntropyLoss()  # Assuming multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 1000

In [5]:
# Training loop
for epoch in range(n_epochs):  # Number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


C:\IT\anaconda3\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1, Loss: 1.728801581005302
Epoch 2, Loss: 1.5829401273079722
Epoch 3, Loss: 1.5334641966942602
Epoch 4, Loss: 1.5019491059439523
Epoch 5, Loss: 1.4821675800886311
Epoch 6, Loss: 1.4662986454974851
Epoch 7, Loss: 1.4487976025641662
Epoch 8, Loss: 1.4386228942759423
Epoch 9, Loss: 1.432696030625694
Epoch 10, Loss: 1.4266839543885315
Epoch 11, Loss: 1.4175965470508334
Epoch 12, Loss: 1.4070550074901178
Epoch 13, Loss: 1.4072199920301416
Epoch 14, Loss: 1.3995031997805736
Epoch 15, Loss: 1.3923272865996708
Epoch 16, Loss: 1.3883633231111656
Epoch 17, Loss: 1.3830772088338954
Epoch 18, Loss: 1.383513257430923
Epoch 19, Loss: 1.377201878494066
Epoch 20, Loss: 1.3785842497957395
Epoch 21, Loss: 1.369271192394319
Epoch 22, Loss: 1.3718879750517547
Epoch 23, Loss: 1.3651250888368844
Epoch 24, Loss: 1.3678689932655674
Epoch 25, Loss: 1.362212347760971
Epoch 26, Loss: 1.3586626320867963
Epoch 27, Loss: 1.3518094050521314
Epoch 28, Loss: 1.3583005881700358
Epoch 29, Loss: 1.3555551044276504


In [8]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
loader = test_loader
# loader = train_loader
results = []
results_x = []
criterion = nn.CrossEntropyLoss() 

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        outputs = model(inputs)
        results.append(outputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get the predicted class (highest log-probability)
        x, predicted = torch.max(outputs, 1)
        results_x.append((x, predicted))
        
        # print(outputs)
        # Calculate the number of correct predictions
        correct += (predicted == labels.argmax(dim=1)).sum().item()  # labels.argmax(dim=1) for one-hot encoded labels
        total += labels.size(0)

# Calculate average loss and accuracy
avg_test_loss = test_loss / len(loader)
accuracy = correct / total * 100

print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')


Test Loss: 1.1849, Test Accuracy: 53.09%


In [25]:
results_x

[(tensor([-6.5484e-04, -8.1538e-01, -4.7827e-04, -4.9135e-01, -7.4117e-01,
          -4.1964e-01, -8.2496e-01, -6.0348e-01, -1.2153e+00, -9.5400e-01,
          -2.2187e-01, -1.1991e-02, -1.0737e+00, -1.1345e+00, -1.2075e-03,
          -4.2074e-01, -5.6550e-03, -1.4195e+00, -3.9013e-01, -1.0258e+00,
          -6.8798e-01, -8.3966e-01, -8.8595e-01, -1.0810e+00, -8.2226e-01,
          -4.6659e-01, -1.1365e+00, -4.3930e-02, -1.1526e+00, -8.1432e-01,
          -4.7135e-01, -2.0064e-03, -9.6952e-01, -8.3103e-01, -1.0798e-02,
          -5.0635e-01, -9.8217e-01, -6.7467e-01, -9.1014e-01, -1.0901e+00,
          -8.0565e-04, -1.1948e+00, -2.5822e-03, -8.4242e-01, -2.4506e-04,
          -1.7021e-03, -8.8014e-01, -1.0128e+00, -1.2652e+00, -4.0420e-03,
          -1.8199e-03, -5.9337e-01, -8.3482e-01, -1.0430e+00, -9.6519e-01,
          -1.1471e+00, -1.2616e+00, -2.7833e-01, -4.7737e-01, -1.2679e+00,
          -4.3825e-01, -3.9098e-01, -8.5959e-01, -1.3095e+00], device='cuda:0'),
  tensor([4, 2, 2, 

In [14]:
# torch.save(model.state_dict(), "model/model_CNN_L.t")
torch.save(model.state_dict(), "model/model_CNN_L_B.t")

In [49]:
loaded_model = NetDropout().to(device)
loaded_model.load_state_dict(torch.load("model/model_CNN_L.t"))
# loaded_model = NetDropoutBig().to(device)
# loaded_model.load_state_dict(torch.load("model/model_CNN_L_B.t"))

<All keys matched successfully>